In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path_to_data = 'data'
path_to_metadata = 'metadata'

path_to_wgi_data = f'{path_to_data}/wgi'
path_to_wgi_metadata = f'{path_to_metadata}/wgi'

path_to_cpi_data = f'{path_to_data}/cpi'
path_to_cpi_metadata = f'{path_to_metadata}/cpi'

In [3]:
wgi_paths = {
    'country_metadata': f'{path_to_wgi_data}/country_metadata.csv',
    'series_metadata': f'{path_to_wgi_data}/series_metadata.csv',
    'wgi_sample_data': f'{path_to_wgi_data}/wgi_sample_data.csv',
    'wgi_sample_metadata': f'{path_to_wgi_data}/wgi_sample_metadata.csv'
}

cpi_paths = {
    'global_scores': f'{path_to_cpi_data}/global_scores.csv',
    'historical_scores': f'{path_to_cpi_data}/historical_scores.csv',
    'time_series': f'{path_to_cpi_data}/time_series.csv'
}

paths = {**wgi_paths, **cpi_paths}

In [4]:
datasets = {}

for name, path in paths.items():
    df = pd.read_csv(path, header=0)
    df_numeric = df.select_dtypes(include=[np.number]).fillna(0)
    df_object = df.select_dtypes(include=[object]).fillna('')
    df = pd.concat([df_object, df_numeric], axis=1)
    datasets[name] = df

In [5]:
def explore_data(name, metadata_path='', detail=False, write_to_file=True):
    """
    Explore the dataset and save the metadata to a json file.
    """
    data = datasets[name]
    result = {
        'Dataset': name,
        'Shape': data.shape,
        'Columns': data.columns.tolist()
    }

    if detail:
        result.update({
            'Head': data.head().to_dict(),
            'Tail': data.tail().to_dict(),
            'Describe': data.describe().to_dict(),
        })

    if write_to_file and metadata_path != '':
        metadata_path = Path(metadata_path)
        metadata_path.mkdir(parents=True, exist_ok=True)
        
        with open(Path(metadata_path, f'{name}.json'), 'w') as f:
            json.dump(result, f, indent=4)
    else:
        print(json.dumps(result, indent=4))

In [6]:
wgi_sample_cols = datasets['wgi_sample_data'].columns.tolist()
wgi_indicators = [
    "Control of Corruption: Estimate",
    "Government Effectiveness: Estimate",
    "Rule of Law: Estimate"
]
cpi_columns = [
    "Country / Territory",
    "ISO3",
    "CPI score",
    "CPI Score",
    "Rank",
    "Year",
    "Region"
]
country_metadata_cols = [
    "Code",
    "Long Name",
    "Income Group",
    "Region"
]

relevant_columns = {
    'country_metadata': country_metadata_cols,
    'wgi_sample_data': wgi_sample_cols,
    'wgi_sample_metadata': wgi_sample_cols,
    'global_scores': cpi_columns,
    'historical_scores': cpi_columns,
    'time_series': cpi_columns
}

In [7]:
def subset(data, cols):
    """
    Given a dataframe `data`, subset it to the columns that contain any of the `substrings`.
    """
    matched_columns = [col for col in data.columns if any(sub in col for sub in cols)]
    return data[matched_columns]

In [8]:
for name, dataset in datasets.items():
    if name in relevant_columns:
        dataset_copy = subset(dataset, relevant_columns[name])
        dataset_copy = dataset_copy.drop('IMD World Competitiveness Yearbook', axis=1, errors='ignore')
        datasets[name] = dataset_copy

In [9]:
for name in wgi_paths.keys():
    explore_data(name, metadata_path=path_to_wgi_metadata, detail=True)
for name in cpi_paths.keys():
    explore_data(name, metadata_path=path_to_cpi_metadata, detail=True)

In [10]:
# WGI

# Combine WGI dataframes and remove duplicates
print(f'Sample data shape: {datasets["wgi_sample_data"].shape}\n'
      f'Sample metadata shape: {datasets["wgi_sample_metadata"].shape}')

wgi_data = datasets['wgi_sample_data'].append(datasets['wgi_sample_metadata'])
wgi_data = wgi_data.drop_duplicates()

datasets['wgi_data'] = wgi_data
explore_data('wgi_data', metadata_path=path_to_wgi_metadata, detail=True)

print(f'Combined WGI data shape: {datasets["wgi_data"].shape}')

Sample data shape: (7704, 15)
Sample metadata shape: (7704, 15)
Combined WGI data shape: (7704, 15)


In [11]:
# CPI data

# clean up column names
cpi_data = datasets['time_series']
cpi_data.columns = cpi_data.columns.str.replace('CPI score', 'CPI Score')

cpi_cols = ', '.join(cpi_data.columns.tolist())
wgi_cols = ', '.join(wgi_data.columns.tolist())

print(f'Shape:\n'
      f'  CPI: {cpi_data.shape}\n'
      f'  WGI: {wgi_data.shape}\n'
      f'Columns:\n'
      f'  CPI: {cpi_cols}\n'
      f'  WGI: {wgi_cols}\n')

Shape:
  CPI: (181, 22)
  WGI: (7704, 15)
Columns:
  CPI: Country / Territory, ISO3, Region, CPI Score 2023, Rank 2023, CPI Score 2022, Rank 2022, CPI Score 2021, Rank 2021, CPI Score 2020, Rank 2020, CPI Score 2019, Rank 2019, CPI Score 2018, Rank 2018, CPI Score 2017, Rank 2017, CPI Score 2016, CPI Score 2015, CPI Score 2014, CPI Score 2013, CPI Score 2012
  WGI: Country Name, Country Code, Series Name, Series Code, 2022 [YR2022], 2021 [YR2021], 2020 [YR2020], 2019 [YR2019], 2018 [YR2018], 2017 [YR2017], 2016 [YR2016], 2015 [YR2015], 2014 [YR2014], 2013 [YR2013], 2012 [YR2012]



In [12]:
merged_data = cpi_data.merge(wgi_data, left_on='Country / Territory', right_on='Country Name')
merged_cols = ', '.join(wgi_data.columns.tolist())

In [13]:
print(f'Shape:\n'
      f'  Merged dataset: {merged_data.shape}\n'
      f'Columns:\n'
      f'  Merged dataset: {merged_cols}\n')

Shape:
  Merged dataset: (5724, 37)
Columns:
  Merged dataset: Country Name, Country Code, Series Name, Series Code, 2022 [YR2022], 2021 [YR2021], 2020 [YR2020], 2019 [YR2019], 2018 [YR2018], 2017 [YR2017], 2016 [YR2016], 2015 [YR2015], 2014 [YR2014], 2013 [YR2013], 2012 [YR2012]



In [14]:
datasets['merged_data'] = merged_data
explore_data('merged_data', metadata_path=f'{path_to_metadata}/merged', detail=True)

In [15]:
# Save the merged data to a csv file
merged_data_path = Path(f'{path_to_data}/merged_data')
merged_data_path.mkdir(parents=True, exist_ok=True)

merged_data.to_csv(f'{merged_data_path}/merged_data.csv', index=False)